In [62]:
### Download renewables in new buildings data for Landkreise
### from inkar.de and compile into csv with Regions as rows
### and years as columns.

In [63]:
# Data URL https://inkar.de, start new query and select
# "Bauen und Wohnen"->"Baulandmarkt und Bautätigkeit" -> "Fertiggestellte Wohngebäude mit erneuerbarer Heizenergie"
# select all Landkreise and all years, download as .csv

In [9]:
# import necessary libraries

import pandas as pd
import numpy as np

In [65]:
#load data
nb_df = pd.read_csv("data/inkar_wohngebaeude_wohnungen_erneuerbare.csv", delimiter=';', header=[0,1])

#drop unnecessary columns and column level
nb_df.drop(columns=["Aggregat","Fertiggestellte Wohnungen mit erneuerbarer Heizenergie"], inplace=True)
nb_df.columns = nb_df.columns.droplevel()

#rename columns and set AGS id as index
nb_df.rename({"Unnamed: 0_level_1": "AGS","Unnamed: 1_level_1":"Name"}, axis=1,inplace=True)
nb_df.set_index('AGS', inplace=True)

nb_df

/home/lalon/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/pandas/core/generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


,Name,2016,2017,2018,2019
AGS,,,,,
1001,"Flensburg, Stadt","7,92","7,69","11,42","3,13"
1002,"Kiel, Stadt","5,00","13,89","16,44","18,99"
1003,"Lübeck, Stadt","17,97","16,15","23,71","12,41"
1004,"Neumünster, Stadt","0,00","11,76","16,36","24,53"
1051,Dithmarschen,"30,70","33,18","37,23","23,87"
...,...,...,...,...,...
16073,Saalfeld-Rudolstadt,"46,25","47,37","47,14","54,17"
16074,Saale-Holzland-Kreis,"45,90","65,75","67,07","55,68"
16075,Saale-Orla-Kreis,"42,86","39,53","55,56","50,00"


In [66]:
#function to convert from German number format (1.000,5) to python number format (1000.5)
def make_column_numeric(se):
    #se = se.str.replace('.','') #not necessary since percentages do not have thousands delimiter
    se = se.str.replace(',','.')
    se = pd.to_numeric(se)
    return se

In [67]:
#convert number format in numeric columns
nb_df[['2016','2017','2018','2019']] = nb_df[['2016','2017','2018','2019']].apply(make_column_numeric)

#prepend column names with indicator name
nb_df = nb_df.add_prefix('newbuilding_renewables_')
nb_df.rename(columns={'newbuilding_renewables_Name':'Name'},inplace=True)
nb_df

,Name,newbuilding_renewables_2016,newbuilding_renewables_2017,newbuilding_renewables_2018,newbuilding_renewables_2019
AGS,,,,,
1001,"Flensburg, Stadt",7.92,7.69,11.42,3.13
1002,"Kiel, Stadt",5.00,13.89,16.44,18.99
1003,"Lübeck, Stadt",17.97,16.15,23.71,12.41
1004,"Neumünster, Stadt",0.00,11.76,16.36,24.53
1051,Dithmarschen,30.70,33.18,37.23,23.87
...,...,...,...,...,...
16073,Saalfeld-Rudolstadt,46.25,47.37,47.14,54.17
16074,Saale-Holzland-Kreis,45.90,65.75,67.07,55.68
16075,Saale-Orla-Kreis,42.86,39.53,55.56,50.00


In [68]:
#save dataframe to file
nb_df.to_csv("data/final_data_landkreis_newbuildings_renewables.csv")

In [11]:
## Dataset: regionalstatistik

#helper function to move values around 
#needed because of structural problems with the raw data
def move_to_third_section(df,title,value):
    second_section = "2_" + title;
    third_section = "3_" + title;

    if "Merkmal" in title:
        df[third_section] = np.where((df[second_section] == value), df[second_section],df[third_section])
        #df[second_section] = np.where((df[second_section] == value),"WOHNGEBAUDE GESAMT",df[third_section])

    elif "Auspraegung" in title:
        merkmal = second_section.replace("Auspraegung","Merkmal")
        df[third_section] = np.where((df[merkmal] == value), df[second_section],df[third_section])
        #df[second_section] = np.where((df[merkmal] == value),"WOHNGEBAUDE GESAMT",df[third_section])

    else:
        print("unknown title: ", title)

    return df

#preprocess dataset
def preprocess_dataset_kreise(df):

    #set AGS Germany to 0 (so we only have numvers)
    df["1_Auspraegung_Code"] = df["1_Auspraegung_Code"].replace({"DG":"0"})    
    df["1_Auspraegung_Code"] = pd.to_numeric(df["1_Auspraegung_Code"])

    #clean up multi level format of raw data 
    df["2_Auspraegung_Label"] = df["2_Auspraegung_Label"].replace({"Insgesamt":'Wohnungen Insgesamt'})  
    #df["2_Auspraegung_Label"] = df["2_Auspraegung_Label"].str.replace('Insgesamt','Wohnungen Insgesamt',regex=True)
    df.loc[df["2_Auspraegung_Code"] == "INSGESAMT","2_Auspraegung_Label"] = "Wohngebäude Insgesamt"

    #rename titles
    titles = ["Merkmal_Code","Merkmal_Label","Auspraegung_Code","Auspraegung_Label"]

    for title in titles:
        if "Code" in title:
            df = move_to_third_section(df,title,"BAUPHE")
        elif "Label" in title:
            df = move_to_third_section(df,title,"Primär verwendete Heizenergie")

    for title in reversed(titles):
        title = "2_" + title
        df[title] = np.where((df["2_Merkmal_Code"] == "BAUPHE"),"Wohngebäude Insgesamt",df[title])

    #rename columns
    df.columns = df.columns.str.replace("1_", "Ort_")
    df.columns = df.columns.str.replace("2_", "Wohnort_")
    df.columns = df.columns.str.replace("3_", "Heizenergie_")


    #make building counts numeric
    building_counts = ["BAU017__Wohngebaeude_insgesamt__Anzahl","WOHN04__Wohnungen_in_Wohngebaeuden_(ohne_Wohnheime)__Anzahl","BAU018__Nichtwohngebaeude_insgesamt__Anzahl"]
    
    for building in building_counts:
        df[building] = df[building].str.replace('-','0',regex=True)
        df[building].fillna(0,inplace=True)
        df[building] = pd.to_numeric(df[building])


    #set total number of buildings for each year and place
    #needed to calculate proportions
    years = list(range(2016, 2021))
    orte = df["Ort_Auspraegung_Code"].unique()

    for ort in orte:
        for year in years:
            isLocation = (df["Ort_Auspraegung_Code"] == ort)
            isYear = (df["Zeit"] == year)

            gesamt = df.loc[isLocation & isYear & (df["Heizenergie_Auspraegung_Code"] == "INSGESAMT")].copy()
            
            gesamt_wohngebaeude = gesamt.loc[(gesamt["Wohnort_Auspraegung_Label"] == "Wohngebäude Insgesamt"),"BAU017__Wohngebaeude_insgesamt__Anzahl"]
            gesamt_wohnungen = gesamt.loc[(gesamt["Wohnort_Auspraegung_Label"] == "Wohnungen Insgesamt"),"WOHN04__Wohnungen_in_Wohngebaeuden_(ohne_Wohnheime)__Anzahl"]

            if(gesamt_wohngebaeude.size == 1 and gesamt_wohnungen.size == 1):
                df.loc[isLocation & isYear,"Wohngebäude_Gesamt"] = int(gesamt_wohngebaeude.item())
                df.loc[isLocation & isYear,"Wohnungen_Gesamt"] = int(gesamt_wohnungen.item())
            else:
                print("error: gesamt is wrong",gesamt)

    #calculate proportions
    df["Wohngebäude_Anteil"] = df["BAU017__Wohngebaeude_insgesamt__Anzahl"] / df["Wohngebäude_Gesamt"] * 100
    df["Wohnungen_Anteil"] = df["WOHN04__Wohnungen_in_Wohngebaeuden_(ohne_Wohnheime)__Anzahl"] / df["Wohnungen_Gesamt"] * 100

    #Drop unneccessary columns to save space
    df.drop(['Statistik_Label','Ort_Merkmal_Label'], axis=1, inplace = True)

    return df

#create pivoted data table
#         viz : df dataframe slice of your data, for example a specific landkreis
#        asProportion : bool, optional
#            if true, the visualisation will use proportion. otherwise, the total number is used. default is true
#        useWohnungen : bool, optional
#            if true, flats will be counted. otherwise, buildings will be counted (which maybe have multiple flats in them). default is true
def create_pivot_tables(df_p,asProportion = True,useWohnungen = True):

    df = df_p.copy()
    
    #rename columns
    df.rename({"Zeit": "year","BAU017__Wohngebaeude_insgesamt__Anzahl":"buildings","Heizenergie_Auspraegung_Label":"energy",
        "Wohngebäude_Anteil":"buildings_anteil","Wohnungen_Anteil":"flats_anteil",
        "WOHN04__Wohnungen_in_Wohngebaeuden_(ohne_Wohnheime)__Anzahl":"flats"}, axis=1,inplace=True)

    # dont add gesamtenergie to plot
    df = df[df["energy"] != "Wohngebäude Insgesamt"]
    df = df[df["energy"] != "Wohnungen Insgesamt"]
    df = df[df["energy"] != "Insgesamt"]

    if useWohnungen:
        column="flats"
        
        #for wohnungen: do not differ between 1, 2 or 3 or more flats
        df= df[df["Wohnort_Auspraegung_Label"] == "Wohnungen Insgesamt"]
    
    else:
        column='buildings'
        df= df[df["Wohnort_Auspraegung_Label"] == "Wohngebäude Insgesamt"]
    
    if asProportion:
        column += "_anteil"

    #pivot data
    orte = df["Ort_Auspraegung_Code"].unique()

    pivot_total = pd.DataFrame()

    for ort in orte:
        snippet = df[df["Ort_Auspraegung_Code"] == ort]

        temp = pd.pivot_table(snippet, index='year',columns='energy', values=column)
        temp = temp.reset_index(level=0)

        temp["ort_ags"] = ort
        temp["ort_name"]= str(snippet["Ort_Auspraegung_Label"].iloc[0])

        pivot_total = pd.concat([pivot_total, temp])

    #sum up renewables and fossils
    renewables = ["Solarthermie","Geothermie","Umweltthermie (Luft/Wasser)","Holz","Biogas/Biomethan","Sonstige Biomasse","Keine Energie (einschl. Passivhaus)"]
    fossils = ["Gas","Strom","Öl","Fernwärme/Fernkälte","Sonstige Heizenergie"]

    pivot_total["renewables"] = pivot_total[renewables].sum(axis=1)
    pivot_total["fossils"] = pivot_total[fossils].sum(axis=1)


    
    pivot_total.to_csv("data/newbuildings_pivot_"+column+".csv")
    return pivot_total



In [15]:
#https://www.regionalstatistik.de/genesis/online?operation=previous&levelindex=3&step=2&titel=Tabellenaufbau&levelid=1655207886087&levelid=1655207857464#abreadcrumb
#Fertigstellungen neuer Wohngebäude und Nichtwohngebäude sowie Wohnungen in Wohngebäuden nach Zahl der Wohnungen und vorwiegender Art der Beheizung - Jahressumme
# downloaded as "flat"

nb_kreise = pd.read_csv("data/newbuildings_kreise.csv",delimiter=";", encoding='latin1')

#preprocessing
nb_kreise = preprocess_dataset_kreise(nb_kreise)
nb_kreise


/var/folders/6w/mz49x3ds0b18vg8p9jx7wg0r0000gn/T/ipykernel_21788/3377774509.py:5: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  nb_kreise = pd.read_csv("data/newbuildings_kreise.csv",delimiter=";", encoding='latin1')


,Statistik_Code,Zeit_Code,Zeit_Label,Zeit,Ort_Merkmal_Code,Ort_Auspraegung_Code,Ort_Auspraegung_Label,Wohnort_Merkmal_Code,Wohnort_Merkmal_Label,Wohnort_Auspraegung_Code,...,Heizenergie_Merkmal_Label,Heizenergie_Auspraegung_Code,Heizenergie_Auspraegung_Label,BAU017__Wohngebaeude_insgesamt__Anzahl,WOHN04__Wohnungen_in_Wohngebaeuden_(ohne_Wohnheime)__Anzahl,BAU018__Nichtwohngebaeude_insgesamt__Anzahl,Wohngebäude_Gesamt,Wohnungen_Gesamt,Wohngebäude_Anteil,Wohnungen_Anteil
0,31121,JAHR,Jahr,2020,KREISE,0,Deutschland,Wohngebäude Insgesamt,Wohngebäude Insgesamt,Wohngebäude Insgesamt,...,Primär verwendete Heizenergie,BAUPHE00,Keine Energie (einschl. Passivhaus),111,0,12960,112935.0,261124.0,0.098287,0.000000
1,31121,JAHR,Jahr,2020,KREISE,0,Deutschland,Wohngebäude Insgesamt,Wohngebäude Insgesamt,Wohngebäude Insgesamt,...,Primär verwendete Heizenergie,BAUPHE02,Öl,776,0,272,112935.0,261124.0,0.687121,0.000000
2,31121,JAHR,Jahr,2020,KREISE,0,Deutschland,Wohngebäude Insgesamt,Wohngebäude Insgesamt,Wohngebäude Insgesamt,...,Primär verwendete Heizenergie,BAUPHE03,Gas,44037,0,5014,112935.0,261124.0,38.993226,0.000000
3,31121,JAHR,Jahr,2020,KREISE,0,Deutschland,Wohngebäude Insgesamt,Wohngebäude Insgesamt,Wohngebäude Insgesamt,...,Primär verwendete Heizenergie,BAUPHE04,Strom,1606,0,780,112935.0,261124.0,1.422057,0.000000
4,31121,JAHR,Jahr,2020,KREISE,0,Deutschland,Wohngebäude Insgesamt,Wohngebäude Insgesamt,Wohngebäude Insgesamt,...,Primär verwendete Heizenergie,BAUPHE05,Fernwärme/Fernkälte,9053,0,1648,112935.0,261124.0,8.016115,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174845,31121,JAHR,Jahr,2016,KREISE,16077,"Altenburger Land, Kreis",GEBWH1,Gebäude nach Anzahl der Wohnungen,WHGZHL03UM,...,Primär verwendete Heizenergie,BAUPHE09,Holz,0,0,0,58.0,101.0,0.000000,0.000000
174846,31121,JAHR,Jahr,2016,KREISE,16077,"Altenburger Land, Kreis",GEBWH1,Gebäude nach Anzahl der Wohnungen,WHGZHL03UM,...,Primär verwendete Heizenergie,BAUPHE10,Biogas/Biomethan,0,0,0,58.0,101.0,0.000000,0.000000
174847,31121,JAHR,Jahr,2016,KREISE,16077,"Altenburger Land, Kreis",GEBWH1,Gebäude nach Anzahl der Wohnungen,WHGZHL03UM,...,Primär verwendete Heizenergie,BAUPHE11,Sonstige Biomasse,0,0,0,58.0,101.0,0.000000,0.000000
174848,31121,JAHR,Jahr,2016,KREISE,16077,"Altenburger Land, Kreis",GEBWH1,Gebäude nach Anzahl der Wohnungen,WHGZHL03UM,...,Primär verwendete Heizenergie,BAUPHE12,Sonstige Heizenergie,0,0,0,58.0,101.0,0.000000,0.000000


In [16]:
# create_pivot_tables_
#         viz : df dataframe slice of your data, for example a specific landkreis
#        asProportion : bool, optional
#            if true, the visualisation will use proportion. otherwise, the total number is used. default is true
#        useWohnungen : bool, optional
#            if true, flats will be counted. otherwise, buildings will be counted (which maybe have multiple flats in them). default is true
pivot = create_pivot_tables(nb_kreise,False,True)
pivot

energy,year,Biogas/Biomethan,Fernwärme/Fernkälte,Gas,Geothermie,Holz,Insgesamt,Keine Energie (einschl. Passivhaus),Solarthermie,Sonstige Biomasse,Sonstige Heizenergie,Strom,Umweltthermie (Luft/Wasser),Öl,ort_ags,ort_name,renewables,fossils
0,2016,919,42350,110439,12661,11690,221451,123,1094,1135,911,1310,37200,1619,0,Deutschland,64822,156629
1,2017,1025,43993,105864,12543,13210,228789,133,1127,1152,821,1243,46018,1660,0,Deutschland,75208,153581
2,2018,693,50748,98898,13934,14300,238317,192,1488,829,1686,1569,52501,1479,0,Deutschland,83937,154380
3,2019,963,55745,98586,14778,12562,246163,174,1157,857,1352,1841,56646,1502,0,Deutschland,87137,159026
4,2020,1104,60312,98726,15368,12026,261124,129,1077,1084,1041,2424,66481,1352,0,Deutschland,97269,163855
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2016,0,3,77,6,0,101,0,1,1,0,0,13,0,16077,"Altenburger Land, Kreis",21,80
1,2017,0,3,31,6,4,68,0,2,1,0,0,21,0,16077,"Altenburger Land, Kreis",34,34
2,2018,0,5,26,9,2,72,0,0,0,0,6,24,0,16077,"Altenburger Land, Kreis",35,37
3,2019,0,110,24,3,1,158,0,1,0,0,1,17,1,16077,"Altenburger Land, Kreis",22,136
